# Proposta

Um notebook que mostre, via dataframes, as listagens de dados da tabela fato e dos agregados. O notebook deve estar previamente executado de modo que se possa ver o conteúdo dos dataframes.

# Model Fato

![fato](../banco/fato.png)

# Model Agregados

![agregados](../banco/agregado.png)

# Implementação

## MySQL

In [1]:
from conexao_mysql import MySQL

In [2]:
mysql_obj = MySQL('localhost', 'alunos', 'alunos', 'atividade02')

Connection to MySQL DB successful


## Listagens

In [3]:
import pandas as pd

In [4]:
def criar_df(query):
    descricao, fatos_list = mysql_obj.execute_read_query_with_description(query)
    nomes_colunas = [d[0] for d in descricao]
    df = pd.DataFrame.from_records(fatos_list, columns=nomes_colunas)
    return df

### Fatos

#### Orçamento Despesas

In [5]:
def criar_fato_df():
    select = """
    SELECT
            forc.valor_orcado, forc.valor_liquidado, tmp.ano_mes, ac.nome acao_nome,
            pg.nome programa_nome, sf.nome subfuncao_nome, fn.nome funcao_nome,
            uo.nome unidade_orcamentaria_nome, os.nome orgao_subordinado_nome, 
            osp.nome orgao_superior_nome
    FROM
        fato_orcamento forc
            INNER JOIN 
        tempo tmp 
            ON tmp.id = forc.tempo_id
            INNER JOIN 
        acao ac 
            ON ac.cod = forc.acao_cod
            INNER JOIN 
        programa pg 
            ON pg.cod = ac.programa_cod
            INNER JOIN 
        subfuncao sf 
            ON sf.cod = forc.subfuncao_cod
            INNER JOIN 
        funcao fn 
            ON fn.cod = sf.funcao_cod
            INNER JOIN 
        unidade_orcamentaria uo 
            ON uo.cod = forc.unidade_orcamentaria_cod
            INNER JOIN 
        orgao_subordinado os 
            ON os.cod = uo.orgao_subordinado_cod
            INNER JOIN 
        orgao_superior osp 
            ON osp.cod = os.orgao_superior_cod
    """
    return criar_df(select)

In [6]:
fato = criar_fato_df()

In [7]:
fato

,valor_orcado,valor_liquidado,ano_mes,acao_nome,programa_nome,subfuncao_nome,funcao_nome,unidade_orcamentaria_nome,orgao_subordinado_nome,orgao_superior_nome
0,0.0,0.0,2019/01,"PROCESSO LEGISLATIVO, FISCALIZACAO E REPRESENT...",ATUACAO LEGISLATIVA DO SENADO FEDERAL,Ação legislativa,Legislativa,SENADO FEDERAL,Fundação Universidade de Brasília,Ministério da Educação
1,52649.9,0.0,2019/01,FISCALIZACAO DA APLICACAO DOS RECURSOS PUBLICO...,CONTROLE EXTERNO,Controle externo,Legislativa,TRIBUNAL DE CONTAS DA UNIAO,Ministério da Economia - Unidades com vínculo ...,Ministério da Economia
2,0.0,0.0,2019/01,APRECIACAO E JULGAMENTO DE CAUSAS NO SUPREMO T...,PRESTACAO JURISDICIONAL DO SUPREMO TRIBUNAL FE...,Ação judiciária,Judiciária,SUPREMO TRIBUNAL FEDERAL,Fundação Universidade de Brasília,Ministério da Educação
3,0.0,0.0,2019/01,JULGAMENTO DE CAUSAS NA JUSTICA FEDERAL,PRESTACAO JURISDICIONAL NA JUSTICA FEDERAL,Ação judiciária,Judiciária,JUSTICA FEDERAL DE PRIMEIRO GRAU,Ministério da Economia - Unidades com vínculo ...,Ministério da Economia
4,0.0,0.0,2019/01,JULGAMENTO DE CAUSAS E GESTAO ADMINISTRATIVA N...,GESTAO DO PROCESSO ELEITORAL,Administração geral,Ciência e Tecnologia,TRIBUNAL SUPERIOR ELEITORAL,Comando do Exército,Ministério da Defesa
...,...,...,...,...,...,...,...,...,...,...
61238,0.0,0.0,2019/12,SUBVENCAO ECONOMICA PARA OPERACOES DECORRENTES...,AGROPECUARIA SUSTENTAVEL,Abastecimento,Agricultura,REC. SOB SUPERV. SEC.TES.NAC. MIN. ECONOMIA,Ministério da Economia - Unidades com vínculo ...,Ministério da Economia
61239,0.0,0.0,2019/12,SUBVENCAO ECONOMICA EM OPERACOES DE FINANCIAME...,OPERACOES ESPECIAIS: OUTROS ENCARGOS ESPECIAIS,Outros encargos especiais,Encargos especiais,RECURSO SOB SUPERVISAO MINISTERIO ECONOMIA,Ministério da Economia - Unidades com vínculo ...,Ministério da Economia
61240,0.0,0.0,2019/12,COMPENSACAO AO FUNDO DO REGIME GERAL DE PREVID...,OPERACOES ESPECIAIS: OUTROS ENCARGOS ESPECIAIS,Outros encargos especiais,Encargos especiais,RECURSO SOB SUPERVISAO MINISTERIO ECONOMIA,Ministério da Economia - Unidades com vínculo ...,Ministério da Economia
61241,0.0,0.0,2019/12,INDENIZACOES E RESTITUICOES RELATIVAS AO PROGR...,OPERACOES ESPECIAIS: OUTROS ENCARGOS ESPECIAIS,Outros encargos especiais,Encargos especiais,RECURSO SOB SUPERVISAO MINISTERIO ECONOMIA,Ministério da Economia - Unidades com vínculo ...,Ministério da Economia


### Agregados

#### Total por Órgão Superior por Ano

In [8]:
def criar_ag_orgao_superior_df():
    select = """
    SELECT
        agos.orgao_superior_cod, os.nome, agos.valor_orcado, agos.valor_liquidado, temp.ano
    FROM
        orgao_superior os
            INNER JOIN
        ag_orgao_superior_ano agos
            ON os.cod = agos.orgao_superior_cod
            INNER JOIN 
        dim_tempo temp
            ON agos.dim_tempo_id = temp.id
    """
    return criar_df(select)

In [9]:
ag_orgao_superior = criar_ag_orgao_superior_df()
ag_orgao_superior

,orgao_superior_cod,nome,valor_orcado,valor_liquidado,ano
0,-1,Sem informação,0.000000e+00,0.000000e+00,2019
1,3000,Tribunal de Contas da União,6.317990e+05,0.000000e+00,2019
2,20000,Presidência da República,1.201180e+10,1.590440e+10,2019
3,22000,"Ministério da Agricultura, Pecuária e Abastec",1.507550e+10,1.575840e+10,2019
4,24000,"Ministério da Ciência, Tecnologia, Inovações",9.344120e+09,9.742210e+09,2019
5,25000,Ministério da Economia,4.817850e+12,5.674450e+12,2019
6,26000,Ministério da Educação,1.045430e+12,1.078730e+12,2019
7,30000,Ministério da Justiça e Segurança Pública,2.414500e+11,8.493530e+10,2019
8,32000,Ministério de Minas e Energia,1.535700e+11,2.272650e+11,2019
9,33000,Ministério da Previdência Social,9.612980e+13,3.658700e+12,2019


#### Total por Função por Ano

In [10]:
def criar_ag_funcao_df():
    select = """
    SELECT
        agfn.funcao_cod, fn.nome, agfn.valor_orcado, agfn.valor_liquidado, temp.ano
    FROM
        funcao fn
            INNER JOIN
        ag_funcao_ano agfn
            ON fn.cod = agfn.funcao_cod
            INNER JOIN 
        dim_tempo temp
            ON agfn.dim_tempo_id = temp.id
    """
    return criar_df(select)

In [11]:
ag_funcao = criar_ag_funcao_df()
ag_funcao

,funcao_cod,nome,valor_orcado,valor_liquidado,ano
0,-1,Sem informação,0.000000e+00,0.000000e+00,2019
1,1,Legislativa,6.317990e+05,2.914090e+06,2019
2,2,Judiciária,0.000000e+00,1.410870e+07,2019
3,3,Essencial à justiça,2.210980e+10,7.870880e+09,2019
4,4,Administração,9.998300e+11,2.970810e+11,2019
5,5,Defesa nacional,5.403500e+12,5.433300e+11,2019
6,6,Segurança pública,2.142700e+11,6.608460e+10,2019
7,7,Relações exteriores,5.892740e+11,3.707140e+10,2019
8,8,Assistência social,1.130560e+11,1.796190e+11,2019
9,9,Previdência social,9.715230e+13,4.158800e+12,2019


#### Total por Programa por Ano

In [12]:
def criar_ag_programa_df():
    select = """
    SELECT
        agpg.programa_cod, pg.nome, agpg.valor_orcado, agpg.valor_liquidado, temp.ano
    FROM
        programa pg
            INNER JOIN
        ag_programa_ano agpg
            ON pg.cod = agpg.programa_cod
            INNER JOIN 
        dim_tempo temp
            ON agpg.dim_tempo_id = temp.id
    """
    return criar_df(select)

In [13]:
ag_programa = criar_ag_programa_df()
ag_programa

,programa_cod,nome,valor_orcado,valor_liquidado,ano
0,-1,Sem informação,0.000000e+00,0.000000e+00,2019
1,1001,GESTÃO DA POLÍTICA DE TURISMO,0.000000e+00,0.000000e+00,2019
2,1008,INCLUSÃO DIGITAL,0.000000e+00,0.000000e+00,2019
3,1025,PROMOÇÃO DA SUSTENTABILIDADE DE ESPAÇOS SUB-RE...,0.000000e+00,0.000000e+00,2019
4,1027,PREVENÇÃO E PREPARAÇÃO PARA DESASTRES,0.000000e+00,0.000000e+00,2019
...,...,...,...,...,...
264,911,OPERACOES ESPECIAIS - REMUNERACAO DE AGENTES F...,3.099160e+09,1.469720e+09,2019
265,913,OPERACOES ESPECIAIS - INTEGRALIZACAO DE COTAS ...,1.396260e+09,3.866560e+09,2019
266,9989,MOBILIDADE URBANA,0.000000e+00,0.000000e+00,2019
267,999,RESERVA DE CONTINGENCIA,0.000000e+00,0.000000e+00,2019


#### Total por Órgão por Ano

In [14]:
def criar_ag_orgao_subordinado_df():
    select = """
    SELECT
        agosb.orgao_subordinado_cod, osb.nome, agosb.valor_orcado, agosb.valor_liquidado, temp.ano
    FROM
        orgao_subordinado osb
            INNER JOIN
        ag_orgao_subordinado_ano agosb
            ON osb.cod = agosb.orgao_subordinado_cod
            INNER JOIN 
        dim_tempo temp
            ON agosb.dim_tempo_id = temp.id
    """
    return criar_df(select)

In [15]:
ag_orgao_subordinado = criar_ag_orgao_subordinado_df()
ag_orgao_subordinado

,orgao_subordinado_cod,nome,valor_orcado,valor_liquidado,ano
0,-3,,0.000000e+00,0.000000e+00,2019
1,3000,Tribunal de Contas da União - Unidades com vín...,6.317990e+05,0.000000e+00,2019
2,20101,Presidência da República,7.869420e+09,1.039450e+10,2019
3,20116,Fundo de Imprensa Nacional,2.553050e+08,1.783420e+08,2019
4,20122,Secretaria Nacional de Políticas para Mulheres,0.000000e+00,0.000000e+00,2019
...,...,...,...,...,...
248,74918,Fundo de Desenvolvimento do Nordeste,0.000000e+00,0.000000e+00,2019
249,74919,Fundo de Desenvolvimento do Centro-Oeste,0.000000e+00,0.000000e+00,2019
250,81000,"Ministério da Mulher, Família e Direitos Human...",2.383490e+08,2.054570e+08,2019
251,90000,Reserva de Contingência,0.000000e+00,0.000000e+00,2019


#### Total por Órgão e Programa por Ano

In [16]:
def criar_ag_orgao_subordinado_programa_df():
    select = """
    SELECT
        agospa.valor_orcado, agospa.valor_liquidado, agospa.orgao_subordinado_cod, agospa.programa_cod,
        osb.nome as orgao_subordinado_nome, pg.nome as programa_nome, temp.ano
    FROM
        ag_orgao_subordinado_programa_ano agospa
            INNER JOIN 
        orgao_subordinado osb 
            ON agospa.orgao_subordinado_cod = osb.cod
            INNER JOIN 
        dim_tempo temp 
            ON agospa.dim_tempo_id = temp.id
            INNER JOIN 
        programa pg 
            ON agospa.programa_cod = pg.cod;
    """
    return criar_df(select)

In [17]:
ag_orgao_subordinado_programa = criar_ag_orgao_subordinado_programa_df()
ag_orgao_subordinado_programa

,valor_orcado,valor_liquidado,orgao_subordinado_cod,programa_cod,orgao_subordinado_nome,programa_nome,ano
0,0.0,0.0,-3,2012,,FORTALECIMENTO E DINAMIZACAO DA AGRICULTURA FA...,2019
1,0.0,0.0,-3,2015,,FORTALECIMENTO DO SISTEMA UNICO DE SAUDE (SUS),2019
2,0.0,0.0,-3,2016,,POLITICAS PARA AS MULHERES: PROMOCAO DA IGUALD...,2019
3,0.0,0.0,-3,2017,,AVIACAO CIVIL,2019
4,0.0,0.0,-3,2021,,"CIENCIA, TECNOLOGIA E INOVACAO",2019
...,...,...,...,...,...,...,...
2910,224017.0,224017.0,81000,901,"Ministério da Mulher, Família e Direitos Human...",OPERACOES ESPECIAIS: CUMPRIMENTO DE SENTENCAS ...,2019
2911,132000.0,66000.0,81000,909,"Ministério da Mulher, Família e Direitos Human...",OPERACOES ESPECIAIS: OUTROS ENCARGOS ESPECIAIS,2019
2912,869600.0,869600.0,81000,910,"Ministério da Mulher, Família e Direitos Human...",OPERACOES ESPECIAIS: GESTAO DA PARTICIPACAO EM...,2019
2913,0.0,0.0,90000,999,Reserva de Contingência,RESERVA DE CONTINGENCIA,2019
